In [1]:
import pandas as pd
from pathlib import Path
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text
import folium

In [2]:
# # create engine to the database

# engine = create_engine("sqlite:///Philly_Bridges")
# query = "SELECT * FROM bridges"

# #Extract the data from the database and covert it to a pandas data frame

# bridge_df = pd.read_sql_query(query, engine)

# bridge_df

In [3]:
bridge_df = pd.read_csv("Cleaned_Bridge_df.csv")
bridge_df

,Bridge_ID,Latitude,Longitude,Year_Built,Bridge_Condition,Average Daily_Traffic,Deck_Width,Structure_Length,SqFt,Repair_Cost_Est,Replacement_Cost_Est,Address
0,452997,40.103450,-74.961092,1990,Good,9729,13.0,18.3,237.90,"$74,700.60","$109,852.70","Dunksferry Road, Mechanicsville, 19020"
1,466574,40.017092,-75.211428,1928,Fair,14596,19.2,32.9,631.68,"$198,347.52","$291,684.56","Ridge Avenue, Wissahickon, 19127"
2,466580,39.963467,-75.186778,1964,Poor,27042,16.1,167.0,2688.70,"$844,251.80","$1,241,534.11","Spring Garden Street, Mantua, 19104"
3,466581,39.963861,-75.185025,1964,Poor,27042,16.2,27.1,439.02,"$137,852.28","$202,721.88","2917, Spring Garden Street, Mantua, 19104"
4,466617,40.035356,-75.208878,1957,Fair,28301,20.7,115.2,2384.64,"$748,776.96","$1,101,131.37","Henry Avenue, 19127"
...,...,...,...,...,...,...,...,...,...,...,...,...
153,471580,39.958900,-75.170372,2017,Good,8800,19.8,29.9,592.02,"$185,894.28","$273,371.16","North 19th Street, Center City, 19132"
154,471996,39.959400,-75.173906,2018,Good,13020,20.4,39.0,795.60,"$249,818.40","$367,376.26","Vine Street Expressway, Center City, 19103"
155,471998,39.959500,-75.175500,2018,Good,12400,25.5,32.9,838.95,"$263,430.30","$387,393.55","North 22nd Street, Center City, 19103"
156,472287,39.960194,-75.159694,1892,Poor,150,4.8,17.1,82.08,"$25,773.12","$37,901.26","Noble Street, Chinatown, Center City, 19123"


In [4]:
# Create a map centered on Philly and with circles that reflect each bridges average daily traffic

myMap = folium.Map(location = (39.952583, -75.165222), tiles = "OpenStreetMap", zoom_start=11.5)

for index, row in bridge_df.iterrows():
    folium.Circle(
        location=[row["Latitude"], row["Longitude"]], 
        radius=[row["Average Daily_Traffic"]/90],
        color="blue", 
        fill=True, 
        fill_opacity=.1, 
        fill_color='lightblue').add_to(myMap)


myMap

In [5]:
# Add my map to the map with the bridge conditon markers

for index, row in bridge_df.iterrows():
    folium.Circle(
        location=[row["Latitude"], row["Longitude"]], 
        radius=[row["Average Daily_Traffic"]/90],
        color="blue", 
        fill=True, 
        fill_opacity=.1, 
        fill_color='lightblue').add_to(myMap)

# Create a function to assign different colors for the three different condition grades    

condition = bridge_df["Bridge_Condition"]
def get_marker_color(condition):
    if condition == "Good":
        return "green"
    elif condition == "Fair":
        return "orange"
    elif condition == "Poor":
        return "darkred"
    else:
        return "gray"

#Create separate groups for each marker type
good_group = folium.FeatureGroup(name="Good (G)")
fair_group = folium.FeatureGroup(name="Fair (F)")
poor_group = folium.FeatureGroup(name="Poor (P)")

#Markers
for index, row in bridge_df.iterrows():
    ID = row["Bridge_ID"]
    lat = row["Latitude"]
    long = row["Longitude"]
    condition = row["Bridge_Condition"]
    address = row["Address"]
    repair_cost = row["Repair_Cost_Est"]
    replace_cost = row["Replacement_Cost_Est"]
    #Marker Coloring
    marker_color = get_marker_color(condition)
    text = f"ID: {ID} <br>Address: {address} <br>Latitude: {lat} <br>Longitude: {long} <br> Condition: {condition} <br> Estimaed Cost to Repair: {repair_cost} <br> Estimaed Cost to Replace: {replace_cost}"
    popup = folium.Popup(text, max_width = 300)
    marker = folium.Marker([lat, long], popup=popup, icon=folium.Icon(color=marker_color))
    
    if condition == "Good":
        marker.add_to(good_group)
    elif condition == "Fair":
        marker.add_to(fair_group)
    elif condition == "Poor":
        marker.add_to(poor_group)
    else:
        marker.add_to(myMap)

good_group.add_to(myMap)
fair_group.add_to(myMap)
poor_group.add_to(myMap)


folium.LayerControl().add_to(myMap)

myMap